In [88]:
import pandas as pd
import numpy as np

In [122]:
users = pd.read_csv('data_design/users.csv', parse_dates=['signup_date'])
events = pd.read_csv('data_design/events.csv', parse_dates=['event_time'])


In [90]:
users.head()

,user_id,signup_date,engagement_score,segment,feature_flag
0,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07,0.263755,new,control
1,f7b1849e-f477-45c6-98e2-b1df3848b346,2024-09-04,0.156159,new,control
2,0e3eff2c-dc61-40a1-9c80-1a2cfaeb2b85,2024-09-08,0.360460,returning,SmartFeed_v2
3,0a2845bd-ae7f-45b9-8182-fdf0b43c5160,2024-09-05,0.118169,new,control
4,77f1e5cc-e90a-46ed-9a02-dcd11759418a,2024-09-07,0.391942,returning,SmartFeed_v2


In [91]:
events.head()

,user_id,event_time,event_name,session_id,feature_flag,content_id
0,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07 00:00:00,session_start,61156f90-187d-412a-b245-02a932e90120,control,NaN
1,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07 00:29:00,card_view,61156f90-187d-412a-b245-02a932e90120,control,card_96
2,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07 00:12:00,card_view,61156f90-187d-412a-b245-02a932e90120,control,card_261
3,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07 00:35:00,session_end,61156f90-187d-412a-b245-02a932e90120,control,NaN
4,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-08 00:00:00,session_start,64290d26-ff75-4018-ac47-231c7deba28e,control,NaN


In [92]:
exposure = (

    events[events['event_name'] == 'session_start'].
    groupby('user_id', as_index = False).
    agg(
        feature_flag = ('feature_flag', 'first'),
        exposure_time = ('event_time', 'min')
    )
)

In [93]:
sessions = events[events['event_name'] == 'session_start'].copy()

In [94]:
sessions.head()

,user_id,event_time,event_name,session_id,feature_flag,content_id
0,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07,session_start,61156f90-187d-412a-b245-02a932e90120,control,NaN
4,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-08,session_start,64290d26-ff75-4018-ac47-231c7deba28e,control,NaN
7,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-09,session_start,e036fdf1-93c5-4bef-b7f0-41b721e53271,control,NaN
12,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-10,session_start,badafd90-5bc2-4684-9ce0-6e8eb5b41ad7,control,NaN
15,f7b1849e-f477-45c6-98e2-b1df3848b346,2024-09-04,session_start,410668b9-9a8c-4a58-91dd-66d87bba5a63,control,NaN


In [95]:
exposure

,user_id,feature_flag,exposure_time
0,0000b368-8df6-4429-8401-d471207017ad,SmartFeed_v2,2024-09-01
1,000527fa-8f64-4cdf-b068-6f565d542395,SmartFeed_v2,2024-09-04
2,0005be36-0dd0-4034-8cec-cabf3cbd9947,SmartFeed_v2,2024-09-06
3,0005e648-cba5-4feb-a321-d50cc0141b61,control,2024-09-05
4,00076a02-3811-4a8e-a19c-2e1312886e26,SmartFeed_v2,2024-09-02
...,...,...,...
49995,fff6bf11-b2ee-4551-8ac1-4a95216af912,control,2024-09-10
49996,fff74585-056b-4311-b9f2-01c462968189,SmartFeed_v2,2024-09-10
49997,fff76877-af68-41a2-8d9e-b189280c3a03,control,2024-09-10
49998,fffddcb4-7cff-430d-8aee-ef554f4ac5e4,SmartFeed_v2,2024-09-10


In [96]:
sessions = (
    sessions.merge(
        exposure,
        on = 'user_id',
        how = 'left'
    )
)

In [97]:
sessions

,user_id,event_time,event_name,session_id,feature_flag_x,content_id,feature_flag_y,exposure_time
0,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07,session_start,61156f90-187d-412a-b245-02a932e90120,control,NaN,control,2024-09-07
1,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-08,session_start,64290d26-ff75-4018-ac47-231c7deba28e,control,NaN,control,2024-09-07
2,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-09,session_start,e036fdf1-93c5-4bef-b7f0-41b721e53271,control,NaN,control,2024-09-07
3,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-10,session_start,badafd90-5bc2-4684-9ce0-6e8eb5b41ad7,control,NaN,control,2024-09-07
4,f7b1849e-f477-45c6-98e2-b1df3848b346,2024-09-04,session_start,410668b9-9a8c-4a58-91dd-66d87bba5a63,control,NaN,control,2024-09-04
...,...,...,...,...,...,...,...,...
800663,3efe1d71-2c3c-473d-b814-ec33ced89d3b,2024-09-21,session_start,287df261-90f2-4c73-9f3a-e4826673396e,control,NaN,control,2024-09-03
800664,3efe1d71-2c3c-473d-b814-ec33ced89d3b,2024-09-22,session_start,7feac57f-734e-4079-a161-5a7e8632e643,control,NaN,control,2024-09-03
800665,3efe1d71-2c3c-473d-b814-ec33ced89d3b,2024-09-23,session_start,56fc62df-a473-4d1f-959b-906f9ae166ee,control,NaN,control,2024-09-03
800666,3efe1d71-2c3c-473d-b814-ec33ced89d3b,2024-09-24,session_start,69ff1f5c-2eb1-4897-844e-2223ca3a2cab,control,NaN,control,2024-09-03


In [98]:
sessions['days_since_first'] = (
    sessions['event_time'] - sessions['exposure_time']
).dt.days

In [99]:
sessions['ret_7d'] = np.where(
    (sessions['days_since_first'] >= 6) & (sessions['days_since_first']<= 8),
    1,
    0
)

In [105]:
exposure = sessions.groupby('user_id').agg(
    feature_flag = ('feature_flag_x', 'first'),
    exposure_time = ('exposure_time', 'min'),
    retained = ('ret_7d', max)
).reset_index()

/var/folders/c1/n3rtprk17jj54dgqz_4lbsyc0000gn/T/ipykernel_12917/3177959337.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  exposure = sessions.groupby('user_id').agg(


In [107]:
exposure.groupby("feature_flag")["retained"].mean()


feature_flag
SmartFeed_v2    0.854897
control         0.853217
Name: retained, dtype: float64

In [113]:
exposure

,user_id,feature_flag,exposure_time,retained
0,0000b368-8df6-4429-8401-d471207017ad,SmartFeed_v2,2024-09-01,1
1,000527fa-8f64-4cdf-b068-6f565d542395,SmartFeed_v2,2024-09-04,0
2,0005be36-0dd0-4034-8cec-cabf3cbd9947,SmartFeed_v2,2024-09-06,1
3,0005e648-cba5-4feb-a321-d50cc0141b61,control,2024-09-05,1
4,00076a02-3811-4a8e-a19c-2e1312886e26,SmartFeed_v2,2024-09-02,0
...,...,...,...,...
49995,fff6bf11-b2ee-4551-8ac1-4a95216af912,control,2024-09-10,1
49996,fff74585-056b-4311-b9f2-01c462968189,SmartFeed_v2,2024-09-10,1
49997,fff76877-af68-41a2-8d9e-b189280c3a03,control,2024-09-10,1
49998,fffddcb4-7cff-430d-8aee-ef554f4ac5e4,SmartFeed_v2,2024-09-10,1


In [114]:
users

,user_id,signup_date,engagement_score,segment,feature_flag
0,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07,0.263755,new,control
1,f7b1849e-f477-45c6-98e2-b1df3848b346,2024-09-04,0.156159,new,control
2,0e3eff2c-dc61-40a1-9c80-1a2cfaeb2b85,2024-09-08,0.360460,returning,SmartFeed_v2
3,0a2845bd-ae7f-45b9-8182-fdf0b43c5160,2024-09-05,0.118169,new,control
4,77f1e5cc-e90a-46ed-9a02-dcd11759418a,2024-09-07,0.391942,returning,SmartFeed_v2
...,...,...,...,...,...
49995,cc3323b0-d02c-4e78-9381-764f9faa73a5,2024-09-06,0.416513,returning,SmartFeed_v2
49996,13ed3ea3-a8a7-4b28-b569-02b2907e592c,2024-09-01,0.352863,returning,SmartFeed_v2
49997,9749b039-c90d-4eed-83e4-ec5caa514477,2024-09-09,0.248595,new,control
49998,8b098874-40ec-47f3-a9ff-3d3a0138b41b,2024-09-03,0.472414,returning,SmartFeed_v2


In [132]:
new = users.merge(
    exposure,
    on = 'user_id',
    how = 'inner'
)

In [133]:
new.head()

,user_id,signup_date,engagement_score,segment,feature_flag_x,feature_flag_y,exposure_time,retained
0,c6a10605-f4bb-4247-a4b8-47a3e4dd2d25,2024-09-07,0.263755,new,control,control,2024-09-07,0
1,f7b1849e-f477-45c6-98e2-b1df3848b346,2024-09-04,0.156159,new,control,control,2024-09-04,0
2,0e3eff2c-dc61-40a1-9c80-1a2cfaeb2b85,2024-09-08,0.360460,returning,SmartFeed_v2,SmartFeed_v2,2024-09-08,1
3,0a2845bd-ae7f-45b9-8182-fdf0b43c5160,2024-09-05,0.118169,new,control,control,2024-09-05,0
4,77f1e5cc-e90a-46ed-9a02-dcd11759418a,2024-09-07,0.391942,returning,SmartFeed_v2,SmartFeed_v2,2024-09-07,1


In [134]:
y = (new["feature_flag_x"] == "SmartFeed_v2").astype(int)
new["feature_flag_x"] = (new["feature_flag_x"] == "SmartFeed_v2").astype(int)
X = pd.get_dummies(new[["segment", "engagement_score"]])


In [135]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
new["propensity"] = model.fit(X, y).predict_proba(X)[:, 1]

In [137]:
treated = new[new["feature_flag_x"] == 1]
control = new[new["feature_flag_x"] == 0]

control = control.iloc[
    np.abs(
        control["propensity"].values[:, None] -
        treated["propensity"].values
    ).argmin(axis=0)
]

matched = pd.concat([treated, control])


In [144]:
matched.groupby("feature_flag_x")["retained"].mean()


feature_flag_x
0    1.000000
1    0.854897
Name: retained, dtype: float64

In [149]:
matched.groupby("segment")["retained"].mean()


segment
new          1.000000
power        0.849810
returning    0.855028
Name: retained, dtype: float64